In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import datetime


plt.rcParams['figure.figsize'] = [40, 40]
%matplotlib inline

In [20]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 15)
data_confirmed = pd.read_csv("./datasets/COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
data_deaths = pd.read_csv("./datasets/COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
data_recovered = pd.read_csv("./datasets/COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

In [21]:
data_confirmed = data_confirmed.groupby('Country/Region').sum()
data_confirmed.drop(['Lat', 'Long'], axis='columns', inplace=True)
data_confirmed = data_confirmed.iloc[:,:].stack().reset_index()
data_confirmed.columns = ['ccse_name', 'date', 'confirmed']

In [22]:
data_confirmed

,ccse_name,date,confirmed
0,Afghanistan,1/22/20,0
1,Afghanistan,1/23/20,0
2,Afghanistan,1/24/20,0
3,Afghanistan,1/25/20,0
4,Afghanistan,1/26/20,0
...,...,...,...
19069,Zimbabwe,4/28/20,32
19070,Zimbabwe,4/29/20,32
19071,Zimbabwe,4/30/20,40
19072,Zimbabwe,5/1/20,40


In [23]:
data_deaths = data_deaths.groupby('Country/Region').sum()
data_deaths.drop(['Lat', 'Long'], axis='columns', inplace=True)
data_deaths = data_deaths.iloc[:,:].stack().reset_index()
data_deaths.columns = ['ccse_name', 'date', 'deaths']
data_deaths

,ccse_name,date,deaths
0,Afghanistan,1/22/20,0
1,Afghanistan,1/23/20,0
2,Afghanistan,1/24/20,0
3,Afghanistan,1/25/20,0
4,Afghanistan,1/26/20,0
...,...,...,...
19069,Zimbabwe,4/28/20,4
19070,Zimbabwe,4/29/20,4
19071,Zimbabwe,4/30/20,4
19072,Zimbabwe,5/1/20,4


In [24]:
data_recovered = data_recovered.groupby('Country/Region').sum()
data_recovered.drop(['Lat', 'Long'], axis='columns')
data_recovered = data_recovered.stack().reset_index()

In [25]:
data_recovered = data_recovered.iloc[2:, :]

In [27]:
data_recovered.columns = ['ccse_name', 'date', 'recovered']

In [28]:
data_recovered.reset_index()
data_recovered

,ccse_name,date,recovered
2,Afghanistan,1/22/20,0.0
3,Afghanistan,1/23/20,0.0
4,Afghanistan,1/24/20,0.0
5,Afghanistan,1/25/20,0.0
6,Afghanistan,1/26/20,0.0
...,...,...,...
19443,Zimbabwe,4/28/20,5.0
19444,Zimbabwe,4/29/20,5.0
19445,Zimbabwe,4/30/20,5.0
19446,Zimbabwe,5/1/20,5.0


In [135]:
data_all = data_confirmed.merge(data_deaths, how='left', 
                               left_on=('ccse_name', 'date'), right_on=('ccse_name', 'date'))
data_all = data_all.merge(data_recovered, how='left',
                         left_on=('ccse_name', 'date'), right_on=('ccse_name', 'date'))
data_all['date'] = pd.to_datetime(data_all.date)
data_all = data_all[(data_all.confirmed != 0) & data_all.recovered != 0]
date_start = pd.to_datetime("2020-04-25")
date_end = pd.to_datetime("2020-05-02")
data_all = data_all[data_all.date > date_start]
data_all['recovered'] = data_all.recovered.astype('int64')

In [136]:
data_all['active_cases'] = data_all.confirmed - data_all.recovered
data_all

,ccse_name,date,confirmed,deaths,recovered,active_cases
95,Afghanistan,2020-04-26,1531,50,207,1324
96,Afghanistan,2020-04-27,1703,57,220,1483
97,Afghanistan,2020-04-28,1828,58,228,1600
98,Afghanistan,2020-04-29,1939,60,252,1687
99,Afghanistan,2020-04-30,2171,64,260,1911
...,...,...,...,...,...,...
19069,Zimbabwe,2020-04-28,32,4,5,27
19070,Zimbabwe,2020-04-29,32,4,5,27
19071,Zimbabwe,2020-04-30,40,4,5,35
19072,Zimbabwe,2020-05-01,40,4,5,35


In [145]:
"""Let's see which countries has more active cases at this day
"""
data_top_active_temp = data_all[data_top_active.date > pd.to_datetime("2020-05-01")].sort_values(by='active_cases', 
                                                                                                        axis='rows', ascending=False)
countries = list(data_top_active_temp.iloc[1:10,:].ccse_name)
countries

e:\vs workspace\ml\.venv\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


['United Kingdom',
 'Italy',
 'France',
 'Russia',
 'Spain',
 'Turkey',
 'Brazil',
 'Netherlands',
 'Belgium']

In [146]:
data_top_active = data_top_active.sort_values(by='date', axis='rows', ascending=False)
data_top_active

,ccse_name,date,confirmed,deaths,recovered,active_cases
17747,US,2020-05-02,1132539,66369,175382,957157
6731,Georgia,2020-05-02,582,8,207,375
1019,Austria,2020-05-02,15558,596,13180,2378
7955,Hungary,2020-05-02,2942,335,625,2317
5099,Djibouti,2020-05-02,1112,2,686,426
...,...,...,...,...,...,...
18659,West Bank and Gaza,2020-04-26,342,2,83,259
12029,Morocco,2020-04-26,4065,161,593,3472
5501,Egypt,2020-04-26,4534,317,1176,3358
11111,Mali,2020-04-26,389,23,112,277


In [147]:
plt.figure(figsize=(20, 10))
plt.title('Countries with most active cases')
days = pd.date_range(date_start+pd.to_timedelta(1, unit='d'), date_end)
for country in countries:
    data = data_top_active[data_top_active.ccse_name == country]
    plt.plot(data.date, data.active_cases, label=country)
plt.legend()
    